# Dataset: https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from keras.layers import Input, Dense
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf

# Set image size and color channels
IMG_SIZE = 128
ColorChannels = 3

# Function to convert video to frames with augmentation
def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)
    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()
        if success and (ID % 7 == 0):
            resized = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
            ImageFrames.append(resized)
        else:
            break
    vidcap.release()
    return ImageFrames

# Paths to video data
PROJECT_DIR = 'D:/Extract'
VideoDataDir = os.path.join(PROJECT_DIR, 'Real Life Violence Dataset')

# Initialize lists to store data
X_original = []
y_original = []

# Select classes and limit the number of videos to process
CLASSES = ["NonViolence", "Violence"]
VIDEO_LIMIT = 350

# Process videos and extract frames
for category in CLASSES:
    path = os.path.join(VideoDataDir, category)
    class_num = CLASSES.index(category)
    for video in os.listdir(path)[:VIDEO_LIMIT]:
        frames = video_to_frames(os.path.join(path, video))
        X_original.extend(frames)
        y_original.extend([class_num] * len(frames))

# Convert lists to NumPy arrays and reshape
X_original = np.array(X_original).reshape(-1, IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_original)

# Split data into training and test sets
stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=73)
for train_index, test_index in stratified_sample.split(X_original, y_original):
    X_train, X_test = X_original[train_index], X_original[test_index]
    y_train, y_test = y_original[train_index], y_original[test_index]

# Reshape data for neural network input
X_train_nn = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

# Load MobileNetV2 model and compile
def load_layers():
    input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, ColorChannels))
    baseModel = MobileNetV2(pooling='avg', include_top=False, input_tensor=input_tensor)
    headModel = Dense(1, activation="sigmoid")(baseModel.output)
    model = Model(inputs=baseModel.input, outputs=headModel)
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    return model

model = load_layers()

# Train the model
model.fit(X_train_nn, y_train, epochs=50, validation_data=(X_test_nn, y_test), batch_size=4)

# Save final model
model.save("final_model.h5")


Epoch 1/50
123/123 [==============================] - 13s 50ms/step - loss: 0.7164 - accuracy: 0.6857 - val_loss: 0.9728 - val_accuracy: 0.7000
Epoch 2/50
123/123 [==============================] - 5s 42ms/step - loss: 0.5340 - accuracy: 0.7796 - val_loss: 5.2432 - val_accuracy: 0.5000
Epoch 3/50
123/123 [==============================] - 5s 42ms/step - loss: 0.5623 - accuracy: 0.7490 - val_loss: 4.9741 - val_accuracy: 0.5000
Epoch 4/50
123/123 [==============================] - 5s 42ms/step - loss: 0.4335 - accuracy: 0.8082 - val_loss: 7.8887 - val_accuracy: 0.5000
Epoch 5/50
123/123 [==============================] - 5s 41ms/step - loss: 0.4564 - accuracy: 0.8143 - val_loss: 4.2900 - val_accuracy: 0.5000
Epoch 6/50
123/123 [==============================] - 5s 41ms/step - loss: 0.3610 - accuracy: 0.8347 - val_loss: 4.5787 - val_accuracy: 0.5000
Epoch 7/50
123/123 [==============================] - 5s 42ms/step - loss: 0.3198 - accuracy: 0.8776 - val_loss: 7.3385 - val_accuracy: 0.504